In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df = spark.read.option("multiline", True).json('./donation_np.json')

In [3]:
df = df.withColumnRenamed("Contribution Mode","mode_of_payment")


In [4]:
df = df.withColumnRenamed("Financial Year","fin_year").withColumnRenamed("PAN Given","pan_given")

In [5]:
from pyspark.sql.functions import sha2,concat_ws

df=df.withColumn("Address", sha2(concat_ws("||", df.Address), 256))

In [6]:
from pyspark.sql.functions import when
df = df.withColumn("mode_of_payment", \
   when((df.mode_of_payment.contains("CASH")), "Cash") \
   .when((df.mode_of_payment.contains("Cheque")),"Cheque")\
   .when((df.mode_of_payment.contains("Bank")),"Bank")\
   .otherwise("Others")\
  )

In [7]:
from pyspark.sql.types import IntegerType
df = df.withColumn("Amount",df["Amount"].cast(IntegerType()))
df.select("Amount").dtypes

Sum = df.groupBy('Party').sum("Amount")
#Sum.show()

Count = df.groupBy('Party').count()
#Count.show()

Average = df.groupBy('Party').avg("Amount")
#Average.show()

Max = df.groupBy('Party').max("Amount")
#Max.show()


In [8]:
for party in Sum.collect():
    df = df.withColumn(party["Party"]+"_SUM_LTD",when((df.Party == party["Party"]),party["sum(Amount)"]).otherwise(0))

#df.select("Party","CPI(M)_SUM_LTD","CPI_SUM_LTD","BJP_SUM_LTD","NCP_SUM_LTD","INC_SUM_LTD").show()


In [9]:
for party in Count.collect():
    df = df.withColumn(party["Party"]+"_COUNT_LTD",when((df.Party == party["Party"]),party["count"]).otherwise(0))

#df.select("Party","CPI(M)_COUNT_LTD","CPI_COUNT_LTD","BJP_COUNT_LTD","NCP_COUNT_LTD","INC_COUNT_LTD").show()


In [10]:
for party in Average.collect():
    df = df.withColumn(party["Party"]+"_AVG_LTD",when((df.Party == party["Party"]),party["Party"]).otherwise(0))


#df.select("Party","CPI(M)_AVG_LTD","CPI_AVG_LTD","BJP_AVG_LTD","NCP_AVG_LTD","INC_AVG_LTD").show()


In [11]:
for eachRecord in Max.collect():
    df = df.withColumn(eachRecord['Party']+'_TOP_DONOR',when((df.Party == eachRecord["Party"]),(df.filter((df.Party == eachRecord['Party']) & (df.Amount == eachRecord['max(Amount)']))).select("Name").collect()[0][0]))

#df.select("INC_TOP_DONOR","CPI_TOP_DONOR","CPI(M)_TOP_DONOR").show()


In [12]:
GroupByOutput = df.groupBy(["Party","fin_year"]).sum("Amount")

#GroupByOutput.show()


In [13]:
for eachRecord in GroupByOutput.collect():
    df = df.withColumn(eachRecord['fin_year']+"_"+eachRecord['Party']+"_SUM",when((df.Party == eachRecord['Party']),eachRecord['sum(Amount)']).otherwise(0))
   
#for each in GroupByOutput.collect():
    #df.select(each["fin_year"]+"_"+each["Party"]+"_SUM").show()


In [14]:
CountOfDonations = df.groupBy("mode_of_payment").count()
#CountOfDonations.show()

In [15]:
#partyList = ['BJP','INC','NCP','CPI','CPI(M)']

for paymentType in CountOfDonations.collect():
    df = df.withColumn(paymentType["mode_of_payment"]+"_count_LTD",when((df.mode_of_payment == paymentType["mode_of_payment"] ),paymentType["count"]).otherwise(0))


In [16]:
#YET TO BE DONE
df.write.format('parquet').save('filename.parquet')

#df.write.parquet("data.parquet")
#df.to_parquet('df.parquet.gzip', compression='gzip')  

Py4JJavaError: An error occurred while calling o641.save.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java